In [1]:
%load_ext autoreload
%autoreload 2

import gc
import os
import pickle
from collections import Counter

import numpy
import pandas
from math import exp, sqrt, log

from keras.models import load_model
from keras import Input, Model
from keras.layers import Lambda, subtract, concatenate
from sklearn.externals import joblib

from importlib import reload
from matplotlib import pyplot as plt
from random import random, shuffle, choice, randint, uniform

from classes import Student, Question
from utils import generate_student_name, create_qs, create_students, generate_attempts, calculate_pass_probability, attempt_q

import random
import math

from NN_utils import BigTable, WeightClip

print("started")

use_saved = True
do_train = True
do_testing = True
create_scorecards = True

base = "../../../isaac_data_files/"

Using TensorFlow backend.


started


In [2]:
# # def fro_norm(w):
# # #     return K.sqrt(K.sum(K.square(K.abs(w))))
# #     return K.sqrt(K.sum(K.square(w)))

# # def cust_reg(w):
# #     m = K.dot(K.transpose(w), w) - np.eye(w.shape)
# #     return fro_norm(m)

# from keras.layers import add, GaussianNoise, Subtract
import tensorflow as tf
# # import tensorflow_probability as tfp
# def tf_cov(x):
#     mean_x = tf.reduce_mean(x, axis=0, keep_dims=True)
#     _, var = tf.nn.moments(x, axes=[0])
#     mx = tf.matmul(tf.transpose(mean_x), mean_x)
#     vx = tf.matmul(tf.transpose(x), x)/tf.cast(tf.shape(x)[0], tf.float32)
#     cov_xx = vx - mx
#     return (cov_xx/var, var, cov_xx)

def tf_corrcoef(x):
    def t(x): return tf.transpose(x)
#     x_t = tf.constant(x)
#     y_t = tf.constant(y)
    xy_t =  x#tf.concat([x, x], axis=0)
    mean_t = tf.reduce_mean(xy_t, axis=0, keep_dims=True)
    cov_t =  tf.matmul(t(xy_t-mean_t),xy_t-mean_t)/(tf.cast(tf.shape(x)[0], tf.float32))
    cov2_t = tf.diag(1/tf.sqrt(tf.diag_part(cov_t)))
    cor = cov2_t @ cov_t @ cov2_t
    return cor

# # def K_corrmx(x):
# #     var = K.var(x, axis=1)

# def tf_corrcoef(x):
#     return tfp.stats.correlation(
#     x,
#     y=None,
#     sample_axis=0,
#     event_axis=-1,
#     keepdims=False,
#     name=None
#     )

# data = numpy.array([[1., 4, 2], 
#                     [5, 6, 24], 
#                     [15, 1, 5], 
#                     [7,3,8], 
#                     [9,4,7]])

# with tf.Session() as sess:
#     print(sess.run(tf_cov(tf.constant(data, dtype=tf.float32))))
#     print(sess.run(tf_corrcoef(tf.constant(data, dtype=tf.float32))))

# ## validating with numpy solution
# pc = numpy.corrcoef(data.T, bias=True)
# print(pc)

In [3]:
from keras.regularizers import l1
from keras.layers import Dropout, multiply, Subtract, Concatenate, Dense
from keras import backend as K
def generate_qs_model(qn_table, psi_table, optimiser):
    psi_sel = Input(shape=(1,), name="psi_select", dtype="int32")
    qn_sel = Input(shape=(1,), name="q_select", dtype="int32")
    print(qn_table, psi_table, psi_sel, qn_sel)
    print("psi_sel shape", psi_sel.shape)

    psi_table.trainable=True
    qn_table.trainable=True
    
    qn_row = qn_table(qn_sel)
    psi_row = psi_table(psi_sel)
    
    qn_ws = qn_table.weights[0]
    psi_ws = psi_table.weights[0]

    klip = Lambda(lambda w: K.clip(w,0,1))
    
    q_masque = klip(qn_row)
#     q_masque = Dropout(0.2)(q_masque)
    s_masque = Lambda(lambda s: K.clip(s,0,1))(psi_row)

#     qn_row = GaussianNoise(.1)(qn_row)
#     psi_row = GaussianNoise(.1)(psi_row)
    
#     L2norm = Lambda(lambda t: K.l2_normalize(t, axis=0))
#     nqn_ws = L2norm(qn_ws)
#     npsi_ws = L2norm(psi_ws)
    
#     L_ortho = Lambda(lambda w : K.sqrt(K.sum(K.square(K.dot(K.transpose(w), w) - numpy.eye(w.shape[1])))))
#     L_ortho = Lambda(lambda w : K.sqrt(K.sum(K.square(K.dot(w, K.transpose(w)) - numpy.eye(w.shape[0])))))

#     L_2 = Lambda(lambda w: K.mean(K.abs(K.sum(K.clip(w,0,1), axis=1)-1))
    L_2 = Lambda(lambda w: K.sqrt(K.mean(K.square(w))))
#     L_2 = Lambda(lambda w: K.mean(K.mean(w, axis=1)))

    svm = Lambda(lambda w: K.sum(w))
    avg = Lambda(lambda w: K.mean(w))
    root = Lambda(lambda w: K.sqrt(w))
    than = Lambda(lambda w: K.tanh(w))
#     ortho_loss = L_ortho(qn_row)
    #L_d1 = Lambda(lambda w: K.mean(100*K.abs(K.clip(K.sum(K.clip(w,0,1), axis=1)-1, -1,1)) ))
#     L_d2 = Lambda(lambda w: K.mean(K.abs(K.sum(K.tanh(w), axis=1)-numpy.tanh(1)) ))

    
    
# ortho_loss = L_ortho(npsi_ws) + L_ortho(nqn_ws)

#      L_2_loss = L_2(qn_ws) + L_2(psi_ws)
#     print("ortho_loss=",ortho_loss)


#     q_w = Lambda(lambda qm: K.sum(qm), name="q_sum")(q_masque)
#     s_w = Lambda(lambda sm: K.sum(sm), name="s_sum")(s_masque)

    qn_row = Lambda(lambda q: K.clip(q,1,math.inf))(qn_row)
    psi_row = Lambda(lambda s: K.clip(s,1,math.inf))(psi_row)


    difs = Concatenate()([psi_row, qn_row])
    h = Dense(100, activation="selu")(difs)
#     h = Dense(30, activation="relu")(h)
#     h = Dense(30, activation="relu")(h)
    Pr = Dense(1, activation="sigmoid")(h)
    
    print("Pr",Pr.shape)
#     model = Model(inputs=[qn_sel, psi_sel], outputs=[Pr,W])
#     model.compile(optimizer=optimiser, loss=["binary_crossentropy","mae"], metrics=["mse","accuracy"], loss_weights=[1,0.1])
    model = Model(inputs=[qn_sel, psi_sel], outputs=Pr)
    model.compile(optimizer=optimiser, loss="binary_crossentropy", metrics=["mse","accuracy"])

    return model

In [4]:
    from keras.optimizers import Adam, SGD
    numpy.random.seed(666)
    import sklearn
    import math


#     minval= 0
    offset = 1
#     maxval = 100
    minval = -math.inf
    maxval = math.inf
    
    beta_min = 0
    beta_max = 100
    theta_min = 0
    theta_max = 100
    
#     minb,maxb,mina,maxa =(1.0, 5.327592330763304, 6.568865477642566, 15.754823559394731)
#     minb,maxb,mina,maxa =(1.0, 4.893499423485679, 6.060544209353401, 16.009594538893413) #100
#     minb,maxb,mina,maxa =(1.0, 3.96, 4.86, 13.36) #50?
#     minb,maxb,mina,maxa =(1.0, 6.744937439353762, 6.560468126136105, 13.903555808574247) #30
    minb,maxb,mina,maxa =(1.0, 3.7812870150169404, 2.691968403956818, 12.165240419546407) # 10?

    dim=10
    inv_sigmoid = lambda p: -numpy.log(1/p - 1)
    p50 = inv_sigmoid( 0.5**(1/dim) )
    p99 = inv_sigmoid( 0.999**(1/dim) )

#     maxval= offset+p99
    av = (minval+maxval)/2
    print("p999", p99)
    print("p50", p50)
    print("max", maxval)
    print("av",av)
    

    n_blobs = 0
    
    n_qs = 100
    n_students = 1000

    n_traits = dim
    nnw = n_traits
    min_active_traits = n_traits
    max_active_traits = n_traits
    
    qs= create_qs(n_qs, minb,maxb, n_traits, min_active_traits, max_active_traits)
    ss= create_students(n_students, mina,maxa, n_traits)
    
    attempts, attempts_by_q, attempts_by_psi, attempts_n_map = generate_attempts(qs,ss)
        
    attempts = numpy.array(attempts)
#     filter_ixs = numpy.random.choice(all_ixs, size=len(attempts), replace=False) #len(attempts)//100, replace=False)
    
    all_ixs = range(len(attempts))
    val_ixs = numpy.random.choice(all_ixs, size=2*int(min(len(attempts)/10,1000)), replace=False) #len(attempts)//100, replace=False)
    trn_ixs = list(set(all_ixs) - set(val_ixs))
    test_ixs = val_ixs[0:len(val_ixs)//2]
    val_ixs = val_ixs[len(val_ixs)//2:]
    
    sz = attempts[trn_ixs, 0]
    qz = attempts[trn_ixs, 1]
    orig_pfz = attempts[trn_ixs, 2]
    pfz = orig_pfz
    print(len(pfz))
#     pfz = numpy.where(orig_pfz < 0.5, 2000, 0).astype("float128")

    vsz = attempts[val_ixs, 0]
    vqz = attempts[val_ixs, 1]
    orig_vpfz = attempts[val_ixs, 2]
    vpfz = orig_vpfz
#     vpfz = numpy.where(orig_vpfz < 0.5, 2000, 0).astype("float128")

    tsz = attempts[test_ixs, 0]
    tqz = attempts[test_ixs, 1]
    tpfz = attempts[test_ixs, 2]


#     s_table =  BigTable((len(uids_across_days), dim), 0, 20, init_hilo= 5)
#     qn_table = BigTable((len(qids_seen),dim), 0, 10, init_hilo= 5-p50)
#     s_table =  BigTable((len(uids_across_days), dim), 0, 24, init_hilo= 9)
#     qn_table = BigTable((len(qids_seen),dim), 0, 14, init_hilo= 9-p50)

p999 9.20979013808
p50 2.63424047381
max inf
av nan
Qinit 0
Q choices are range(0, 10)
Qinit 1
Q choices are range(0, 10)
Qinit 2
Q choices are range(0, 10)
Qinit 3
Q choices are range(0, 10)
Qinit 4
Q choices are range(0, 10)
Qinit 5
Q choices are range(0, 10)
Qinit 6
Q choices are range(0, 10)
Qinit 7
Q choices are range(0, 10)
Qinit 8
Q choices are range(0, 10)
Qinit 9
Q choices are range(0, 10)
Qinit 10
Q choices are range(0, 10)
Qinit 11
Q choices are range(0, 10)
Qinit 12
Q choices are range(0, 10)
Qinit 13
Q choices are range(0, 10)
Qinit 14
Q choices are range(0, 10)
Qinit 15
Q choices are range(0, 10)
Qinit 16
Q choices are range(0, 10)
Qinit 17
Q choices are range(0, 10)
Qinit 18
Q choices are range(0, 10)
Qinit 19
Q choices are range(0, 10)
Qinit 20
Q choices are range(0, 10)
Qinit 21
Q choices are range(0, 10)
Qinit 22
Q choices are range(0, 10)
Qinit 23
Q choices are range(0, 10)
Qinit 24
Q choices are range(0, 10)
Qinit 25
Q choices are range(0, 10)
Qinit 26
Q choices are

500
600
700
800
900
passed 50841/100000
98000


In [10]:
    from keras import regularizers
    opt = None
#     s_table =  BigTable((n_students, dim), -math.inf, math.inf, init_hilo= 10)
#     qn_table = BigTable((n_qs,dim), -math.inf, math.inf, init_hilo= 10-p50)

#     s_table =  BigTable((n_students, dim), -math.inf, math.inf, init_hilo=None)
#     qn_table = BigTable((n_qs,dim), -math.inf, math.inf, init_hilo=None)

    s_table =  BigTable((n_students, dim), 1, math.inf, init_hilo=1)#, regulariser=regularizers.l2(0))
    qn_table = BigTable((n_qs,dim), 1, math.inf, init_hilo=1)#, regulariser=regularizers.l2(0))

    
#     print("s start value = ", offset+av)
#     print("q start value = ", offset+av-p50)

    q_wgts_history = []
    s_wgts_history = []

#     s_table =  BigTable((len(userz), 1), 0, 10)
    qn_table.trainable = True
    s_table.trainable = True
    q_model = generate_qs_model(qn_table, s_table, Adam())
    
#     qn_table.trainable = False
#     s_table.trainable = True
#     s_model = generate_qs_model(qn_table, s_table, Adam())

    psi_ws = s_table.get_weights()[0]
    qn_ws = qn_table.get_weights()[0]
    
    print("PSI WS:")
    print(psi_ws)
    print("QN WS:")
    print(qn_ws)
#     input("start...")
    
#     with tf.Session() as sess:
#         print(sess.run(tf_corrcoef(tf.constant(qn_ws, dtype=tf.float32))))

#     print(numpy.corrcoef(qn_ws.T))#-numpy.eye(qn_ws.shape[1])))

    

#     zer0z = numpy.zeros((len(pfz),dim))
#     vzer0z = numpy.zeros((len(vpfz),dim))
#     qz=list(qz)
#     sz=list(sz)
#     pfz = list(pfz)


<NN_utils.BigTable object at 0x7faddab0fbe0> <NN_utils.BigTable object at 0x7fadda57f3c8> Tensor("psi_select_2:0", shape=(?, 1), dtype=int32) Tensor("q_select_2:0", shape=(?, 1), dtype=int32)
psi_sel shape (?, 1)
kk (100, 10)
selector shape (?, 1)
flat selector shape (?,)
call kk (100, 10)
'rows' shape, (?, 10)
kk (1000, 10)
selector shape (?, 1)
flat selector shape (?,)
call kk (1000, 10)
'rows' shape, (?, 10)
Pr (?, 1)
PSI WS:
[[ 0.77348971  1.99678969  1.7150476  ...,  0.31257534  1.11737061
   1.88809371]
 [ 0.14557195  1.36638618  1.22847533 ...,  1.41593051  0.07025862
   0.18451071]
 [ 0.04691148  1.54577875  0.86273575 ...,  1.53148317  0.81927752
   0.8550005 ]
 ..., 
 [ 1.59669495  0.96086669  1.8748858  ...,  1.95325828  1.84723353
   1.90262079]
 [ 0.42949629  1.42697906  0.70398355 ...,  0.36282992  0.10429454
   1.16826415]
 [ 1.69580889  0.6001482   1.32481909 ...,  1.03118443  1.6791842
   1.80375695]]
QN WS:
[[  2.07602262e-01   6.17479086e-01   1.67932296e+00   2.2768

In [6]:
    from keras.callbacks import EarlyStopping
    
    es = EarlyStopping(monitor="loss", restore_best_weights=True, patience=5)
    initpfz = [0.5 for _ in range(len(attempts))]
 
    for _ in range(10):
        q_model.fit(x=[attempts[:,1], attempts[:,0]], y=numpy.array(initpfz).reshape(-1,1), batch_size=1000, shuffle=True, epochs=1, verbose=1)
#         s_model.set_weights(q_model.get_weights())
#         s_model.fit(x=[attempts[:,1], attempts[:,0]], y=numpy.array(initpfz).reshape(-1,1), batch_size=1000, shuffle=True, epochs=1, verbose=1)
#         q_model.set_weights(s_model.get_weights())


Instructions for updating:
Use tf.cast instead.
Epoch 1/1
100000/100000 [==============================] - 1s 6us/step - loss: 0.7068 - mean_squared_error: 0.0063 - acc: 0.0000e+00
Epoch 1/1
100000/100000 [==============================] - 0s 4us/step - loss: 0.6944 - mean_squared_error: 6.0948e-04 - acc: 0.0000e+00
Epoch 1/1
100000/100000 [==============================] - 0s 4us/step - loss: 0.6934 - mean_squared_error: 1.4005e-04 - acc: 0.0000e+00
Epoch 1/1
100000/100000 [==============================] - 0s 4us/step - loss: 0.6933 - mean_squared_error: 5.1715e-05 - acc: 0.0000e+00
Epoch 1/1
100000/100000 [==============================] - 0s 4us/step - loss: 0.6932 - mean_squared_error: 3.4290e-05 - acc: 0.0000e+00
Epoch 1/1
100000/100000 [==============================] - 0s 4us/step - loss: 0.6932 - mean_squared_error: 2.8640e-05 - acc: 0.0000e+00
Epoch 1/1
100000/100000 [==============================] - 0s 4us/step - loss: 0.6932 - mean_squared_error: 2.4980e-05 - acc: 0.0000e+

In [7]:
init_predz = q_model.predict(x=[vqz, vsz])
for hat,ture in zip(init_predz,vpfz):
    print(hat, ture)
    
print(q_model.evaluate(x=[vqz,vsz], y=vpfz))

[ 0.49193469] 1.0
[ 0.49700302] 0.0
[ 0.49716392] 1.0
[ 0.49552488] 0.0
[ 0.50143427] 0.0
[ 0.49334666] 0.0
[ 0.50348002] 1.0
[ 0.49397776] 0.0
[ 0.49656385] 0.0
[ 0.50743133] 0.0
[ 0.49598455] 0.0
[ 0.49616301] 0.0
[ 0.50599694] 0.0
[ 0.49457324] 1.0
[ 0.50057048] 0.0
[ 0.50156027] 1.0
[ 0.50196022] 1.0
[ 0.49395472] 0.0
[ 0.50310826] 1.0
[ 0.50720197] 0.0
[ 0.49692002] 0.0
[ 0.50252241] 0.0
[ 0.49826449] 0.0
[ 0.49722338] 0.0
[ 0.50596017] 1.0
[ 0.4997198] 1.0
[ 0.49448156] 0.0
[ 0.50151902] 1.0
[ 0.50042999] 0.0
[ 0.502168] 0.0
[ 0.49312255] 1.0
[ 0.49624008] 0.0
[ 0.49548829] 1.0
[ 0.49637377] 0.0
[ 0.50817335] 1.0
[ 0.49874482] 0.0
[ 0.49481997] 0.0
[ 0.51016116] 1.0
[ 0.50369036] 1.0
[ 0.50191593] 0.0
[ 0.49833909] 0.0
[ 0.49882197] 0.0
[ 0.49365437] 0.0
[ 0.49938688] 1.0
[ 0.48545319] 0.0
[ 0.49985072] 0.0
[ 0.49292251] 0.0
[ 0.50496948] 0.0
[ 0.49992874] 0.0
[ 0.49481609] 1.0
[ 0.49581459] 0.0
[ 0.50625336] 1.0
[ 0.50499761] 0.0
[ 0.49996367] 1.0
[ 0.50377315] 1.0
[ 0.49557942]

[ 0.50343961] 1.0
[ 0.5000819] 1.0
[ 0.49865025] 0.0
[ 0.49681726] 1.0
[ 0.49356502] 1.0
[ 0.49230942] 1.0
[ 0.50901389] 1.0
[ 0.4972553] 0.0
[ 0.49377158] 1.0
[ 0.50426728] 1.0
[ 0.49804002] 0.0
[ 0.50168842] 1.0
[ 0.51162958] 1.0
[ 0.51099432] 0.0
[ 0.48830202] 0.0
[ 0.50224847] 0.0
[ 0.49980524] 1.0
[ 0.50359964] 1.0
[ 0.49590388] 1.0
[ 0.50837487] 1.0
[ 0.49407265] 0.0
[ 0.4993909] 1.0
[ 0.4998689] 1.0
[ 0.50375253] 0.0
[ 0.50332403] 1.0
[ 0.5004974] 0.0
[ 0.5034942] 0.0
[ 0.49988356] 0.0
[ 0.49604708] 0.0
[ 0.4939509] 1.0
[ 0.50321656] 1.0
[ 0.50214088] 1.0
[ 0.50598139] 1.0
[ 0.49866584] 1.0
[ 0.50062346] 1.0
[ 0.49695024] 1.0
[ 0.4970617] 1.0
[ 0.50326949] 0.0
[ 0.49301413] 0.0
[ 0.49294585] 0.0
[ 0.49600267] 0.0
[ 0.4972885] 1.0
[ 0.50562668] 1.0
[ 0.50260741] 0.0
[ 0.49854508] 1.0
[ 0.49949947] 0.0
[ 0.49800453] 1.0
[ 0.4985387] 0.0
[ 0.50457788] 0.0
[ 0.49526873] 1.0
[ 0.49969324] 1.0
[ 0.4987312] 1.0
[ 0.49826232] 0.0
[ 0.49562755] 1.0
[ 0.50452667] 0.0
[ 0.49248835] 1.0
[ 0

In [11]:
    wgts = q_model.get_weights()
#     q_model = generate_qs_model(qn_table, s_table, Adam())
#     q_model = generate_qs_model(qn_table, s_table, Adam())
    
    qn_table.trainable = True
    s_table.trainable = True
    q_model = generate_qs_model(qn_table, s_table, Adam())
    q_model.set_weights(wgts)
    
#     qn_table.trainable = False
#     s_table.trainable = True
#     s_model = generate_qs_model(qn_table, s_table, Adam())
    
    min_loss = math.inf
    max_acc = 0
    max_tr_acc = 0
    init_patience = 10
    patience = init_patience
    best_q_weights = None
    best_s_weights = None

    zz = numpy.zeros(len(pfz))
    vzz = numpy.zeros(len(vpfz))
    

<NN_utils.BigTable object at 0x7faddab0fbe0> <NN_utils.BigTable object at 0x7fadda57f3c8> Tensor("psi_select_3:0", shape=(?, 1), dtype=int32) Tensor("q_select_3:0", shape=(?, 1), dtype=int32)
psi_sel shape (?, 1)
selector shape (?, 1)
flat selector shape (?,)
call kk (100, 10)
'rows' shape, (?, 10)
selector shape (?, 1)
flat selector shape (?,)
call kk (1000, 10)
'rows' shape, (?, 10)
Pr (?, 1)


In [131]:
print(len(pfz))

98000


In [12]:
    for it in range(1000):
        print(it)
        psi_ws = s_table.get_weights()[0]
        qn_ws = qn_table.get_weights()[0]
#         preds = qs_model.predict(x=[qz, sz])
#         for prix, p in enumerate(preds[0:100]):
#             print(prix,":",qz.iloc[prix], sz.iloc[prix], p,pfz.iloc[prix])
        
#         h = qs_model.fit(x=[qz, sz], y=pfz, batch_size=256, shuffle=True, epochs=1, verbose=1, validation_split=min(.1, 1000/len(pfz)))
#         print(h.history)
#         loss, mse, acc = qs_model.train_on_batch(x=[qz,sz], y=pfz)
#         val_loss, val_mse, val_acc = qs_model.evaluate(x=[vqz, vsz], y=vpfz)
        h = q_model.fit(x=[qz,sz], y=numpy.array(pfz).reshape(-1,1), batch_size=10, shuffle=True, epochs=1, verbose=1, validation_data=[[vqz,vsz],vpfz])# validation_split=min(.1, 1000/len(pfz)))
#         s_model.set_weights(q_model.get_weights())
#         h = s_model.fit(x=[qz,sz], y=numpy.array(pfz).reshape(-1,1), batch_size=1000, shuffle=True, epochs=1, verbose=2)#, validation_data=[[vsz,vqz],vpfz])# validation_split=min(.1, 1000/len(pfz)))
#         q_model.set_weights(s_model.get_weights())

    
        loss = h.history['loss'][-1]
#         val_loss = loss
#         mse = h.history['add_w_mean_squared_error'][-1]
#         acc = h.history['sPr_prod_acc'][-1]

        mse = 0 #h.history['sPr_prod_mean_squared_error'][-1]

        acc = h.history['acc'][-1]

#         val_acc = acc
#         val_loss = loss
        val_acc = h.history['val_acc'][-1]
        val_loss = h.history['val_loss'][-1]

#         val_acc = h.history['val_sPr_prod_acc'][-1]
#         val_loss = h.history['val_sPr_prod_loss'][-1]
#         mse = h.history['mean_squared_error'][-1]

#         qs_model.train_on_batch(x=[qz,sz], y=pfz)
#         loss,mse,acc = qs_model.evaluate(x=[qz,sz], y=pfz)
#         print(loss, mse, acc)
        print("SUM DCOV", 0.5*(numpy.mean(numpy.abs(numpy.corrcoef(qn_ws.T))-numpy.eye(qn_ws.shape[1])) + numpy.mean(numpy.abs(numpy.corrcoef(psi_ws.T))-numpy.eye(psi_ws.shape[1]))))
        print("Q MAG", (numpy.mean(qn_ws)))
        print("VAL:", val_loss, val_acc)
        
        print("av no of qn components:",numpy.mean(numpy.sum((qn_ws > .5), axis=1)))
        print("av qn component:",numpy.mean(numpy.mean(qn_ws, axis=1)))
#         if val_loss <= min_loss or val_acc > max_acc:
        if max_tr_acc < acc:
            max_tr_acc = acc

        if min_loss >= val_loss:
            print("new min loss")
            min_loss = val_loss
            max_acc = val_acc
            best_q_weights = qn_table.get_weights()
            best_s_weights = s_table.get_weights()
            patience = init_patience
        else:
            patience -= 1
#             qn_table.set_weights(best_q_weights * numpy.random.uniform(low=0.5, high=1.5, size=(len(best_q_weights), dim)))
#             s_table.set_weights(best_s_weights * numpy.random.uniform(low=0.5, high=1.5, size=(len(best_s_weights), dim)))
            
#         if it % 10 == 0:
        q_wgts_history.append( qn_table.get_weights()[0] ) 
        s_wgts_history.append( s_table.get_weights()[0] )
    
        if patience <= 0:
#         else:
            print("patience spent")
            break
        print("patience is now {}".format(patience))

    qn_table.set_weights(best_q_weights)
    s_table.set_weights(best_s_weights)

    print("final:", min_loss, max_tr_acc, max_acc)
    
#10000s testing, default LR, BS=256, no noise no q-gates
# 1 = 64.0
# 2 = 65.2
# 4 = 66.2 
# 8 = 65.2 
# 16= 66.2

0
Train on 98000 samples, validate on 1000 samples
Epoch 1/1
98000/98000 [==============================] - 14s 141us/step - loss: 0.6524 - mean_squared_error: 0.2306 - acc: 0.6071 - val_loss: 0.6023 - val_mean_squared_error: 0.2075 - val_acc: 0.6890
SUM DCOV 0.0476473840081
Q MAG 1.01949
VAL: 0.602342459559 0.689000001252
av no of qn components: 7.63
av qn component: 1.01949
new min loss
patience is now 10
1
Train on 98000 samples, validate on 1000 samples
Epoch 1/1
98000/98000 [==============================] - 13s 133us/step - loss: 0.6137 - mean_squared_error: 0.2130 - acc: 0.6604 - val_loss: 0.6089 - val_mean_squared_error: 0.2107 - val_acc: 0.6490
SUM DCOV 0.127082737809
Q MAG 1.29191
VAL: 0.608935218155 0.649000002444
av no of qn components: 10.0
av qn component: 1.29191
patience is now 9
2
Train on 98000 samples, validate on 1000 samples
Epoch 1/1
98000/98000 [==============================] - 12s 127us/step - loss: 0.6107 - mean_squared_error: 0.2119 - acc: 0.6607 - val_loss: 

In [13]:
from sklearn.metrics import accuracy_score

val_loss, val_mse, val_acc = q_model.evaluate(x=[tqz, tsz], y=tpfz)
print(val_loss, val_mse, val_acc)

# evpfz = (numpy.exp(-vpfz) >= 0.5)
# phat = (numpy.exp(-q_model.predict(x=[vqz, vsz])) >= 0.5)

etpfz = (tpfz >= 0.5)
phat = (q_model.predict(x=[tqz, tsz]) >= 0.5)


for hat,ture in zip(etpfz, phat):
    print(hat, ture)

print(accuracy_score(etpfz, phat))
#30 comp = .581


1000/1000 [==============================] - 0s 34us/step
0.646543523788 0.227174803257 0.644
True [ True]
True [ True]
True [ True]
False [False]
True [False]
False [False]
True [ True]
True [False]
True [False]
True [False]
True [ True]
True [ True]
False [ True]
True [ True]
True [ True]
False [ True]
True [False]
False [False]
False [False]
False [False]
True [False]
False [False]
True [False]
True [ True]
False [ True]
True [False]
False [False]
False [False]
True [ True]
True [ True]
True [ True]
False [False]
False [False]
False [False]
True [False]
False [ True]
False [False]
False [False]
False [ True]
True [False]
False [ True]
True [ True]
True [False]
False [ True]
False [False]
True [ True]
True [False]
True [ True]
True [False]
True [False]
True [ True]
True [False]
True [False]
True [False]
False [False]
False [False]
False [ True]
False [False]
True [False]
False [False]
False [False]
False [False]
True [ True]
True [ True]
False [ True]
True [False]
True [ True]
True [

False [ True]
True [False]
False [False]
False [ True]
False [False]
True [False]
False [False]
False [False]
False [ True]
True [False]
0.644


In [ ]:
    from sklearn.metrics import confusion_matrix
    y_hatz_raw = q_model.predict(x=[vqz, vsz]) 
    print(q_model.evaluate(x=[vqz, vsz], y=vpfz) )
    
    print("TN FP\nFN TP")
    for thresh in [0.3, 0.4, 0.45, 0.5, 0.55, 0.6, 0.75, 0.8, 0.9]:
        y_hatz = (y_hatz_raw >= thresh)
        print("Threshold {}".format(thresh))
        print(len(y_hatz), sum(y_hatz))
        print(confusion_matrix(vpfz, y_hatz),"\n")
        #     ix=0
# #     if last_qn_table:
#     for lw,w in zip(last_qn_table, qn_table.get_weights()[0]):
#         if w != lw:
#             print("{} : {} - {}->{}".format(ix,qids_seen[ix],lw,w))            
#         ix+=1
#     last_qn_table = qn_table.get_weights()[0]

In [ ]:
# D | acc      |w>0       |igate     \rng=20    |corr_reg
# 2k|  
# 1k| .        | .858/807 |
# 5c| .
# 4c| .849/795 | .848/796 | .848/795 |
# 3c| .
# 2c| .
# 1c| .857/771 | .857/772 | .856/765 |
# 50| .844/783
# 30| .845/756 | .845/763 | .852/760 | .850/757 | .833/754
# 20| .839/763
# 10| .834/756 | .823/769 |                     | .815/745
#  5| .818/747 | .-/-
#  1| .771/684 | 
qn_ws = qn_table.get_weights()[0]
psi_ws = s_table.get_weights()[0]
# numpy.set_printoptions(precision=1)
# with tf.Session() as sess:
#     I = numpy.eye(qn_ws.shape[0])
#     print(sess.run(tf.abs(tf_cov(tf.constant(qn_ws, dtype=tf.float32)) -I) ))
#     print(sess.run(tf.reduce_sum(tf.abs(tf_corrcoef(tf.constant(qn_ws, dtype=tf.float32)) -I) )))


history_len = range(len(q_wgts_history))
ww = qn_table.get_weights()[0].shape[0]


# print(len(q_wgts_history))

numpy.set_printoptions(precision=2)
# print(qn_table.get_weights()[0])
# print(s_table.get_weights()[0])
# # for row in s_table.get_weights()[0]:
# #     print(row)
# # print(numpy.argwhere(numpy.isnan(psi_ws)))

# with tf.Session() as sess:
#     print(sess.run(K.sum(tf.abs(tf_corrcoef(tf.constant(qn_ws, dtype=tf.float32)) - numpy.eye(qn_ws.shape[1] )))))
#     print(sess.run(K.sum(tf.abs(tf_corrcoef(tf.constant(psi_ws, dtype=tf.float32)) - numpy.eye(qn_ws.shape[1] )))))

# print(numpy.cov(psi_ws.T))
    
print("q:")
print(numpy.corrcoef(qn_ws.T))
print("Psi:")
print(numpy.corrcoef(psi_ws.T))

In [ ]:
q_pairs = []
q_book_tups = []
book_qixs=[]

min_w = numpy.min(qn_table.get_weights()[0]) #Q weights
max_w = numpy.max(qn_table.get_weights()[0]) #Q weights

print("min_w is",min_w)
print("max_w is",max_w)
print(max_w - min_w)
print(ww)

for wix in range(ww):
#     if (qids_seen[wix].startswith("ch_") or qids_seen[wix].startswith("ch-")):
#         book_qixs.append(wix)
    mags = []
    ncs = []
    for hix in range(len(q_wgts_history)):
        wgts = q_wgts_history[hix][wix] - min_w
        mag = numpy.linalg.norm(wgts) # calculate the Frobinius norm
        mags.append(mag)
    plt.plot(range(len(q_wgts_history)), mags, alpha=0.1)
plt.gcf().set_size_inches(20,5)
plt.show()

In [ ]:
q_pairs = []
raw_wgts = qn_table.get_weights()[0]

min_w = numpy.min(raw_wgts)
wgts = raw_wgts #- min_w

# for rw, w in zip(raw_wgts,wgts):
#     print(sum(rw), sum(w), min(w))

for ix in set(qz):
    wgts_i = wgts[ix] - min_w
#     print(wgts_i)
    prob_qid = qids_seen[ix]
#     mag = numpy.sum(wgts_i) 
    mag = numpy.linalg.norm(wgts_i)
    n_comps = numpy.sum((wgts_i > .1))
    q_pairs.append((ix, prob_qid, mag, n_comps))

q_pairs.sort(key=lambda x: x[2])


hist_data = []
for p in q_pairs:
    hist_data.append(p[2])


dim_data = []
for p in q_pairs:
    dim_data.append(p[3])

    
# # df = pandas.DataFrame(columns=["qid","diff","dims"])
for p in q_pairs:#[0:100]:
    print(p[0], p[1], p[2], p[3])
    if p[0] in unfailed:
        print("SHDN'T BE HERE: UNFAILED")
    if p[0] in unpassed:
        print("SHDN'T BE HERE: UNPASSED")
    
# for p in q_pairs[-100:]:
#     print(p[0], p[1], p[2])

numpy.savetxt("./{}d_all_qs.csv".format(dim), q_pairs , fmt="%s", delimiter=",")
bookpairs=[(p[0],p[1]) for p in q_book_tups]
bookpairs.sort(key=lambda x: x[1])
numpy.savetxt("./{}d_book_qs.csv".format(dim), bookpairs , fmt="%s", delimiter=",")

plt.hist(hist_data)
plt.gcf().set_size_inches(20,5)
plt.show()

print(numpy.mean(dim_data))
plt.hist(dim_data)
plt.gcf().set_size_inches(20,5)
plt.show()

In [ ]:
from random import shuffle

# swgts = s_table.get_weights()[0]
# print("student range:", min(swgts), max(swgts))

uids_across_days = list(uids_across_days)
# uids_across_days = sorted(uids_across_days)

# print(uids_across_days)
# for u in uids_across_days[0:100]:
#     print(u)

xs = range(len(s_wgts_history))
# ww = s_wgts_history[0][0]
print(s_wgts_history[0])

u_pairs = []
unchanged = set()

ww_choices = numpy.random.choice(uids_across_days, size=min(len(uids_across_days),20000), replace=False)
# ww_choices = uids_across_days
print(len(ww_choices))
for uid in ww_choices:
    uix = uids_across_days.index(uid)
    mags = []
    for hix in xs:
        wgts = s_wgts_history[hix][uix] - min_w
        mag = numpy.linalg.norm(wgts) # calculate the Frobinius norm
#         mag = sum(wgts)
        n_comps = numpy.sum(wgts > 0)
        mags.append(mag)
    if mags[0] == mags[-1]:
        unchanged.add(uid)
    else:
        u_pairs.append((uid, mag, n_comps))
        plt.plot(xs,mags, alpha=0.1)
plt.gcf().set_size_inches(20,5)
plt.show()

u_pairs.sort(key=lambda x: x[1])
hist_data = []
for p in u_pairs:
    print(p[0], p[1])
    hist_data.append(p[1])

for p in sorted(u_pairs)[0:100]:
    print(p[0], p[1], p[2])
print("--")
for p in sorted(u_pairs)[-100:]:
    print(p[0], p[1], p[2])
    
plt.hist(hist_data, bins=30)
plt.gcf().set_size_inches(20,5)
plt.show()

In [ ]:
from collections import defaultdict
from datetime import datetime
u_dt_map = defaultdict(dict)
st_hist_data = []
ed_hist_data = []
for p in u_pairs:
    ustr = p[0].split("_")[0]
    dt = datetime.strptime(p[0].split("_")[1].split(" ")[0], "%Y-%m-%d")
    u_dt_map[ustr][dt]=p[1]

# for k in u_dt_map:
#     dts = sorted(u_dt_map[k])
#     st_hist_data.append(u_dt_map[k][dts[0]])
#     ed_hist_data.append(u_dt_map[k][dts[-1]])

kct=0
cols = 5
rows = 10
f,pltz = plt.subplots(rows,cols, sharey=True, sharex=True)
for k in u_dt_map:
#     if len(u_dt_map[k]) < 10:
#         continue
    dts = sorted(u_dt_map[k])
    delta = (max(dts) - min(dts))
    
#     d = datetime(1979,7,16)/
    d = delta.days
    st_hist_data.append(u_dt_map[k][dts[0]])
    ed_hist_data.append(u_dt_map[k][dts[-1]])

    if d < 180:
        continue    
#     print(k,":", dts)
    mgs = [u_dt_map[k][dt] for dt in dts]
#     print(mgs)
#     print(kct, kct//cols, kct%cols)
    if kct<(cols*rows):
        pltz[kct//cols, kct%cols].plot_date(x=dts, y=mgs, label=k, ls="-", fmt="")
    kct+=1
    
plt.gcf().set_size_inches(20,40)
# plt.ylim(0)
plt.show()
    
# print("...")
# for el in unchanged:
#     print(el)
    
st_mean = numpy.mean(st_hist_data)
ed_mean = numpy.mean(ed_hist_data)

plt.hist(st_hist_data, alpha=0.5, label="Before", bins=30)
plt.axvline(numpy.mean(st_mean), color='blue', linestyle='dashed', linewidth=1)
plt.hist(ed_hist_data, alpha=0.5, label="After", bins=30)
plt.axvline(numpy.mean(ed_mean), color='red', linestyle='dashed', linewidth=1)

_, max_ = plt.ylim()
plt.text(st_mean+0.3, max_ - max_/10, 'Start Mean:\n{:.2f}'.format(st_mean))

plt.text(ed_mean+0.3, 
         max_ - max_/5, 
         'End Mean:\n{:.2f}'.format(ed_mean))

plt.gcf().set_size_inches(20,5)
plt.legend()
plt.show()

print(len(st_hist_data))
print(numpy.mean(st_hist_data), numpy.std(st_hist_data))
print(numpy.mean(ed_hist_data), numpy.std(ed_hist_data))
print(len(unchanged), len(u_pairs))

## Student calibration and change over time

### Daily snapshots
For a student s, the student's attempts at questions are partitioned into daily snapshots: {s0, s1, s2 ...}
For the purposes of calibration, student snapshots are mutually independent: that is the scores for sm are conditionally independent of the scores for sn n!=m.  The independence is conditional because information is transmitted via the calibration of the questions encountered on day n: {q}\_n.

Note that since a student cannot pass an item twice, it's impossible for two "passes" on different days to exist.  It is possible for a student to make several failed attempts at an item, and then pass it on a later date.  Since item calibrations hold across all time, but students are recalibrated daily, the student's ability will be calibrated such that it "straddles" the item across this timeframe.

An example of a student's scores over time:

10214_2015-11-09 169.724 20

10214_2015-11-10 192.286 20

10214_2015-11-11 201.752 20

10214_2016-05-10 196.241 20

10214_2016-05-15 202.679 20

10214_2016-05-16 201.018 20

10214_2016-05-20 182.211 20

10214_2016-05-21 184.925 20

10214_2016-06-23 190.13 20



In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
indices = [qids_seen.index(tup[0]) for tup in q_book_tups]
print(indices)

mutate = TSNE(n_components=2)
vqz_set = list(set(vqz))
warr = numpy.array([qn_table.get_weights()[0][vq] for vq in vqz_set])

data_flat = mutate.fit_transform(warr)
# data_flat = warr

# plt.scatter(x=[tvp[2][0] for tvp in q_book_tups], y=[tvp[2][1] for tvp in q_book_tups])
plt.scatter(x=data_flat[:,0], y=data_flat[:,1], alpha=0.5)

for ix,vq in enumerate(vqz_set):
#     qix = qids_seen.index(tup[0])
#     L = tup[1].split("|")[0]
#     L = qids_seen[vq].split("|")[0]
    L = qids_seen[vq].split("|")[0]
    plt.annotate(L, (data_flat[ix,0], data_flat[ix,1]), alpha=0.5 )
plt.gcf().set_size_inches(20,20)
plt.show()


# warr = numpy.array([qn_table.get_weights()[0][bq] for bq in book_qixs])
# data_flat = mutate.fit_transform(warr)
# # data_flat = warr
# # plt.scatter(x=[tvp[2][0] for tvp in q_book_tups], y=[tvp[2][1] for tvp in q_book_tups])
# plt.scatter(x=data_flat[:,0], y=data_flat[:,1], alpha=0.5)

# for ix, bq in enumerate(book_qixs):
#     L = qids_seen[bq].split("|")[0] + "|" + qids_seen[bq].split("|")[1][0:5] 
#     plt.annotate(L, (data_flat[ix,0], data_flat[ix,1]), alpha=0.5 )
# plt.gcf().set_size_inches(20,20)
# plt.show()

In [ ]:
for ix,q in enumerate(qn_table.get_weights()[0]):
    print(ix, qids_seen[ix], q)

In [ ]:
for ix,s in enumerate(s_table.get_weights()[0]):
    print(ix, uids_across_days[ix], s)

In [ ]:
plot_w = 6
plot_h = (1+len(d_hist_data)//plot_w)

# fig, axes = plt.subplots(nrows=(len(d_hist_data)), ncols=1)            
plt.gcf().set_size_inches(3*plot_w,3*plot_h)
for ax_i, d in enumerate(d_hist_data):
    data = d_hist_data[d]
    print(d, len(data))
#     print(min(data), max(data))
#     print(data)
    subplot = plt.subplot(plot_h, plot_w, ax_i+1)
    subplot.hist(data)
plt.show()

print(len(d_hist_data), "days of activity")
print(len(qids_seen), "qids")
print(len(uids_seen), "uids")
print(len(book_qids), "book qids")
print(book_qids)
            
#         for item in attz4date[attz4date["question_id"]==qid].loc[["user_id", "correct"]]:
#             print("\t",item)
        

In [ ]:
print(passes)
plt.hist(passes)
plt.show()

In [ ]:
# assignments.loc[:, "creation_date"] = pandas.to_datetime(assignments["creation_date"]).dt.floor("D")
# pid_override = set()
# for h in ass_summ["hexes"]: # go through all hexes ever assigned...
#     pid_override.update(eval(h))
# pid_override = sorted(list(pid_override))

LOAD_PQIDS=True

if LOAD_PQIDS:
    pid_override = joblib.load(base+"pid_override.pkl")
    all_qids = joblib.load(base+"all_qids.pkl")
    print("loaded pqids....")

else:
    pid_override = set()
    qids_in_play = set()
    ss = set()
    for aix in ass_summ.index:
        ss.update(eval(ass_summ.loc[aix,"students"]))
        # hxz = [h for h in eval(ass_summ.loc[aix,"hexes"]) if (h.startswith("ch_") or h.startswith("ch-i"))]
        # ass_summ.loc[aix, "hexes"] = str(hxz)

    for s in ss:
        attz = get_attempts_from_db(s)
        qids_in_play.update([q for q in list(attz["question_id"])])# if (q.startswith("ch_") or q.startswith("ch-i"))] )
        pid_override.update([s.split("|")[0].replace("-","_") for s in list(attz["question_id"]) if (s.startswith("ch_") or s.startswith("ch-i"))])
    all_qids = sorted(qids_in_play)
    pid_override = sorted(pid_override)
    all_page_ids = pid_override

    print(all_qids)
    print("Qids in play len", len(all_qids))

    print(pid_override)
    print("Pids in play len", len(pid_override))
    joblib.dump(all_qids, base+"all_qids.pkl")
    joblib.dump(pid_override, base+"pid_override.pkl")
    
all_page_ids = list(set([ s.split("|")[0] for s in all_qids ]))
print(all_page_ids)
print(len(all_page_ids))

In [ ]:
import copy
from hwgen.common import get_user_data
start_dates = {}
ss = set()
first_assign = {}
min_ts = None
max_ts = None
for aix in ass_summ.index:
    ts, grid, students = ass_summ.loc[aix,["creation_date","group_id","students"]]
    
    if (not min_ts) or ts < min_ts:
        min_ts = ts
    if (not max_ts) or ts > max_ts:
        max_ts = ts
    
    students = eval(students) # convert str rep to list
    ss.update(students)
    for s in students:
        if s not in first_assign:
            first_assign[s] = copy.copy(ts)
psi_df = get_user_data(list(ss))
for ix in psi_df.index:
    psi, rd = psi_df.loc[ix, ["id","registration_date"]]
    if psi not in start_dates:
        ts = first_assign[psi]
        reg_date = pandas.Timestamp(psi_df.loc[psi_df["id"]==psi,"registration_date"].values[0])
        start_dates[psi] = (ts, reg_date)

for s in start_dates:
    print(s, start_dates[s])

print(min_ts, max_ts)

In [ ]:
BUILD_SXUA = False
# print(ass_summ[0:10])
numpy.set_printoptions(precision=1, suppress=True)
if BUILD_SXUA:
    print("getting start dates")
#     try:
#         start_dates = joblib.load(base + "start_dates.pkl")
#         print("loaded")
#     except:
#         start_dates = build_start_dates(ass_summ)
    if True:
        joblib.dump(start_dates, base+"start_dates.pkl")
        print("built")
    ass_summ = ass_summ[ass_summ["has_book_hexes"] == True]  # discard activity that has nowt to do with the book
#     for psi in list(start_dates.keys())[0:10]:
#         print(psi, start_dates[psi])
    print("building SXUA")
    SXUA = build_SXUA(base, ass_summ, all_qids, pid_override=pid_override, start_dates=start_dates)
    f = open(base + "SXUA.comp.pkl", 'wb')
    pickle.dump(SXUA, f)
    f.close()
    print("compressed and SAVED")
else:
    print("loading SXUA")
    f = open(base + "SXUA.comp.pkl", 'rb')
    SXUA = pickle.load(f)
    print("SXUA loaded with {} entries", len(SXUA))
    f.close()

In [ ]:
print(len(SXUA))
for k in SXUA:
#     print(k)
    tss = SXUA[k]
    print("{}: ({} {}), {} ".format(k, min(tss), max(tss), len(tss)))

In [ ]:
# ass_summ = ass_summ[0:8000]
ass_summ = ass_summ[ass_summ["has_book_hexes"]==True] # discard activity that has nowt to do with the book
# ass_summ = ass_summ[ass_summ["num_hexes"]==1] # discard activity that has nowt to do with the book
print("ass summ pre filtered, shape {}".format(ass_summ.shape))
ass_summ = ass_summ[ass_summ["include"]==True]
print("ass summ post filtered, shape {}".format(ass_summ.shape))

tss = ass_summ.loc[:,"creation_date"]
print(numpy.min(tss), numpy.max(tss))

In [ ]:
# import zlib
# numpy.set_printoptions(precision=1, suppress=True)

# for ix in ass_summ.index[0:100]:
#     aid,stz,ts = ass_summ.loc[ix,["id","students","creation_date"]]
#     stz = eval(stz)
#     print("----Aid =",aid)
#     for psi in stz:
#         psi_sxua = SXUA[psi]
# #         for ts in psi_sxua:
#         S,X,U,A = pickle.loads(zlib.decompress(psi_sxua[ts]))
#         print(aid,psi,ts,S)

In [ ]:
numpy.set_printoptions(precision=4, suppress=True)
aug = augment_data(ass_summ, SXUA, pid_override, filter=True)
aid_list, s_list, x_list, u_list, a_list, y_list, psi_list, hexes_to_try_list, hexes_tried_list, _, gr_id_list, ts_list, maxdops = aug

In [ ]:
# import sys
# i=0
# for aid, psi, sl,xl,ul,al,hxtt,hxtd in zip(aid_list, psi_list, s_list,x_list,u_list,a_list,hexes_to_try_list, hexes_tried_list):
#     print(psi,":",aid,":",sl)
# #     print([x for x in xl])
#     print("...",sum(xl), numpy.nonzero(xl))
#     print([all_qids[ix] for ix in numpy.nonzero(xl)[0]])
#     i += 1
#     if i == 1000:
#         break
# #     print(xl)
# #     print(y_list)

In [ ]:
COUNT_TEACHERS=False
if COUNT_TEACHERS:
    t_test = None
    t_train = None
    ct = Counter()
    for t in numpy.unique(ass_summ["owner_user_id"]):
        t_assignments = ass_summ[ass_summ["owner_user_id"] == t]
        ct[t] = ass_summ.shape[0]
    print(ct.most_common(20))
    print("teachers counted")

    assct = 0
    teacherct = 0
    kept_ts = []
    for (t, tct) in list(ct.most_common(len(ct))):
        t_assignments = ass_summ[ass_summ["owner_user_id"] == t]
        this_t_assts = t_assignments.shape[0]
        if this_t_assts < 5:
            print("not enough assignments",t,tct,this_t_assts)
        else:
            teacherct += 1
            kept_ts.append((t,ct[t]))
    print(teacherct, assct)

    teacherN = teacherct
    test_insts = 0
    target_test = 6000 #teacherN // 10

    t_test = []
    t_train = []
    put_in_test_set = False
    n_train = 0
    n_test = 0
    for (t, tct) in kept_ts:
        num_students = 0
        t_assignments = ass_summ[ass_summ["owner_user_id"] == t]
        print(t,":",t_assignments.shape[0], "new training assts")

        if put_in_test_set and n_test<target_test:
            # test_insts += 1
            print("TEST>>")
            aug = augment_data(t_assignments, SXUA, pid_override=pid_override, filter=True)
            num_students = len(aug[0])
            n_test += num_students
            if len(t_test) == 0:
                t_test = t_assignments
            else:
                t_test = pandas.concat([t_test, t_assignments])
            put_in_test_set = False
        else:
            print("TRAIN>>")
            aug = augment_data(t_assignments, SXUA, pid_override=pid_override, filter=False)
            num_students = len(aug[0])
            n_train += num_students
            if len(t_train) == 0:
                t_train = t_assignments
            else:
                t_train = pandas.concat([t_train, t_assignments])
            put_in_test_set = True

        if n_train >= 50000:
            break

    tr = t_train
    tt = t_test
    print("numbers of (student,asst) pairs:", n_train, n_test)
    tr.to_csv(base + "tr_ttb.csv")
    tt.to_csv(base + "tt_ttb.csv")
else:
    tr = pandas.read_csv(base + "tr_ttb.csv")
    tt = pandas.read_csv(base + "tt_ttb.csv")

print("len t_train, t_test",len(tr), len(tt))
ass_summ = pandas.concat([tr,tt])
print("Split complete!")
print("{} {}".format(len(tt), len(tr)))

In [ ]:
do_x_val = True
xrmses = []
xaccs = []
if do_x_val:
    n_folds = 10
    start_indices = [len(tr)*x//n_folds for x in range(n_folds)]
    print(start_indices)
    fold_size = len(tr)//n_folds
    for start_target in start_indices:
        ttx = tr[start_target:(start_target+fold_size)]        
        trx = tr.drop(tr.index[range(start_target,start_target+fold_size)])
        aug_trx = augment_data(trx, SXUA, pid_override=pid_override, all_qids=all_qids, all_page_ids=all_page_ids, filter=False)
        modelx = train_deep_model(aug_trx)
        aug_ttx = augment_data(ttx, SXUA, pid_override=pid_override, all_qids=all_qids, all_page_ids=all_page_ids, filter=False)
        (rmse,acc) = evaluate3(aug_ttx, modelx, pid_override)
        xrmses.append(rmse)
        xaccs.append(acc)
        

In [ ]:
print(xrmses)
print(xaccs)
print(numpy.mean(xrmses))
print(numpy.mean(xaccs))

In [ ]:
    model = train_deep_model(aug)
    print("...deleted original X,y")
    model.save(base + 'hwg_model.hd5')
    # joblib.dump(xmask, base + 'hwg_xmask.pkl')
    # joblib.dump(sc, base + 'hwg_mlb.pkl')
    # joblib.dump((sscaler,levscaler,volscaler), base + 'hwg_scaler.pkl')
    print("saved model and exit")